### Tutorial for constructing the QUBO matrix found in Quantum Annealing Feature Selection paper [1]

[1] http://1qbit.com/files/white-papers/1QBit-White-Paper-%E2%80%93-Optimal-Feature-Selection-in-Credit-Scoring-and-Classification-Using-a-Quantum-Annealer_-_2017.04.13.pdf

In [27]:
from IPython.display import display, Math, Latex
import numpy as np
from numpyarray_to_latex.jupyter import to_jup
from numpyarray_to_latex import to_ltx


def remove_diagonal(M: np.ndarray) -> np.ndarray:
    M = M[~np.eye(M.shape[0],dtype=bool)].reshape(M.shape[0],-1)
    return M

***Regarding  which correlation measure to choose from:***

It was noted, however, that methods with a ‘‘smooth’’ distribution of coefficients
(Spearman, Pearson, etc.) worked better with the quadratic objective function than correlation methods with sharper
jumps, such as ‘‘mutual information’’ scores (scikit op. cit., Rosenberg 2007). In the end, the Spearman method was
chosen as it is simple and easy to reproduce. However, this is an area where more research is needed.




In [28]:
rng = np.random.default_rng(seed=42)
n_features = 5
n_samples = 10

X = np.ones(n_features) # Feature encoding binary vector 
V = rng.integers(0,2, size=(n_samples,1)) # Example model target outputs 
U = rng.random(size=(n_samples,n_features)) # Example model variables (1 feature per column --> (sample x feature) dimensional matrix)




p_Vj = np.corrcoef(U, V, rowvar=False)[1:,:1] # Correlation with model target outputs
p_ij = np.corrcoef(U,rowvar=False)            # Correlation between the features

p_Vj = np.abs(p_Vj)
p_ij = np.abs(p_ij)


V_ltx= to_ltx(V)
p_Vj_ltx = to_ltx(p_Vj)
p_ij_ltx= to_ltx(p_ij)

In [29]:
Latex(rf""" $V ={V_ltx}$ <br/> <br/> 
            $p_{{Vj}} ={p_Vj_ltx}$ <br/> <br/>
            $p_{{ij}} ={p_ij_ltx}$ <br/> <br/>

        """)



<IPython.core.display.Latex object>

In [30]:
p_Vj_terms_ltx = to_ltx(p_Vj)

Latex(rf"""$p_{{Vj}} ={p_Vj_terms_ltx}$ <br/> <br/>""")

<IPython.core.display.Latex object>

In [31]:
np.fill_diagonal(p_ij,0)


In [32]:
p_ij

array([[0.        , 0.3930328 , 0.34527953, 0.03567888, 0.50229227],
       [0.3930328 , 0.        , 0.07119079, 0.12787199, 0.2202626 ],
       [0.34527953, 0.07119079, 0.        , 0.16156206, 0.27642409],
       [0.03567888, 0.12787199, 0.16156206, 0.        , 0.09691974],
       [0.50229227, 0.2202626 , 0.27642409, 0.09691974, 0.        ]])

In [33]:
s= n_features
n, m = np.shape(p_Vj)
matrix_pVj = np.zeros((s,s))
for i in range(s):
    matrix_pVj[i,i] = p_Vj[i]


matrix_pVj_ltx = to_ltx(matrix_pVj)
Latex(rf"""$p_{{Vj}} ={matrix_pVj_ltx}$ <br/> <br/>""")

/var/folders/4l/b4lbhbs5785_4f4_ms4lf7xr0000gn/T/ipykernel_41074/4112784261.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  matrix_pVj[i,i] = p_Vj[i]


<IPython.core.display.Latex object>

In [34]:
_check = p_ij + matrix_pVj


_check = to_ltx(_check)
Latex(rf"""$p_{{ij}} + p_{{Vj}} ={_check}$ <br/> <br/>""")

<IPython.core.display.Latex object>

In [35]:
# Weighting the independence vs prediction power of variables
alpha = 0.5

Q = (alpha * p_ij) - (1-alpha)*matrix_pVj

Q_ltx = to_ltx(Q)
Latex(rf"""$Q ={Q_ltx} $ """)

<IPython.core.display.Latex object>

In [36]:
Q

array([[-0.1965164 ,  0.1965164 ,  0.17263977,  0.01783944,  0.25114613],
       [ 0.1965164 , -0.17263977,  0.0355954 ,  0.063936  ,  0.1101313 ],
       [ 0.17263977,  0.0355954 , -0.01783944,  0.08078103,  0.13821204],
       [ 0.01783944,  0.063936  ,  0.08078103, -0.25114613,  0.04845987],
       [ 0.25114613,  0.1101313 ,  0.13821204,  0.04845987, -0.06454308]])

In [37]:
np.savetxt(f'QUBO.csv', Q, delimiter=',')

In [38]:
# E is the equation which the optimization process minimizes, @ is an matrix multiplication operator defined in numpy
E = -X.T @ Q @ X

In [39]:
E

np.float64(-1.527829925732705)